In [52]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [57]:
merge=pd.read_csv('merge.csv',index_col=0,thousands = ',')

In [58]:
merge.head()

,zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,h_15,h_16,h_17,h_18,h_19,h_20,h_21,h_22,h_23,h_24
0,1,2004,1,1,16853.0,16450.0,16517.0,16873.0,17064.0,17727.0,...,55.0,55.0,52.0,46.0,40.0,40.0,39.0,38.0,40.0,41.0
1,1,2004,1,2,14155.0,14038.0,14019.0,14489.0,14920.0,16072.0,...,54.0,56.0,57.0,53.0,50.0,47.0,46.0,45.0,45.0,45.0
2,1,2004,1,3,14439.0,14272.0,14109.0,14081.0,14775.0,15491.0,...,69.0,68.0,68.0,65.0,64.0,63.0,62.0,63.0,63.0,62.0
3,1,2004,1,4,11273.0,10415.0,9943.0,9859.0,9881.0,10248.0,...,71.0,72.0,71.0,68.0,67.0,67.0,65.0,64.0,65.0,64.0
4,1,2004,1,5,10750.0,10321.0,10107.0,10065.0,10419.0,12101.0,...,66.0,66.0,66.0,66.0,63.0,54.0,52.0,49.0,47.0,47.0


In [59]:
merge_noNan=merge.dropna()

idx=[i for i in range(len(merge_noNan))]
merge_noNan = merge_noNan.reset_index(drop=True)
merge_noNan.head()

,zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,h_15,h_16,h_17,h_18,h_19,h_20,h_21,h_22,h_23,h_24
0,1,2004,1,1,16853.0,16450.0,16517.0,16873.0,17064.0,17727.0,...,55.0,55.0,52.0,46.0,40.0,40.0,39.0,38.0,40.0,41.0
1,1,2004,1,2,14155.0,14038.0,14019.0,14489.0,14920.0,16072.0,...,54.0,56.0,57.0,53.0,50.0,47.0,46.0,45.0,45.0,45.0
2,1,2004,1,3,14439.0,14272.0,14109.0,14081.0,14775.0,15491.0,...,69.0,68.0,68.0,65.0,64.0,63.0,62.0,63.0,63.0,62.0
3,1,2004,1,4,11273.0,10415.0,9943.0,9859.0,9881.0,10248.0,...,71.0,72.0,71.0,68.0,67.0,67.0,65.0,64.0,65.0,64.0
4,1,2004,1,5,10750.0,10321.0,10107.0,10065.0,10419.0,12101.0,...,66.0,66.0,66.0,66.0,63.0,54.0,52.0,49.0,47.0,47.0


In [60]:
def nans(df):
    return df[df.isnull().any(axis=1)]

nanList = nans(merge)
nanList=list(nanList.index)

In [61]:
zone=[0 for i in range(21)]
for i in range(1,21):
    zone[i]=merge_noNan[merge_noNan['zone_id']==i]

In [62]:
mlr = [[0 for i in range(25)] for i in range(21)]

In [67]:
for i in range(1,21):
    for j in range(1,25):
        if j==1:
            if i<12:
                x=zone[i][['year','month','day','h24','h_24','h_1']]
                y=zone[i]['h1']
                x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8, test_size=0.2)
                mlr[i][j] = LinearRegression()
                mlr[i][j].fit(x_train,y_train)
            else:
                x=zone[i][['year','month','day','h24']]
                y=zone[i]['h1']
                x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8, test_size=0.2)
                mlr[i][j] = LinearRegression()
                mlr[i][j].fit(x_train,y_train)
        else:
            if i<12:
                x=zone[i][['year','month','day','h'+str(j-1),'h_'+str(j-1),'h_'+str(j)]]
                y=zone[i]['h'+str(j)]
                x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8, test_size=0.2)
                mlr[i][j] = LinearRegression()
                mlr[i][j].fit(x_train,y_train)
            else:
                x=zone[i][['year','month','day','h'+str(j-1)]]
                y=zone[i]['h'+str(j)]
                x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8, test_size=0.2)
                mlr[i][j] = LinearRegression()
                mlr[i][j].fit(x_train,y_train)

In [70]:
from tqdm.notebook import tqdm
for nn in tqdm(range(len(nanList))):
    zone_id = merge['zone_id'][nanList[nn]]
    for hour in range(1,25):
        if hour==1:
            if zone_id<=11:
                b=merge.loc[[nanList[nn]-1]][['year','month','day','h24','h_24','h_1']]
                b= b.values.tolist()
#                 b[0][3]=int(str(b[0][3]).replace(',',''))
                my_data = b
                merge['h1'][nanList[nn]]=int(mlr[zone_id][hour].predict(my_data)[0])
            else:
                b=merge.loc[[nanList[nn]-1]][['year','month','day','h24']]
                b= b.values.tolist()
#                 b[0][3]=int(str(b[0][3]).replace(',',''))
                my_data = b
                merge['h1'][nanList[nn]]=int(mlr[zone_id][hour].predict(my_data)[0])
        else:
            if zone_id<=11:
                b=merge.loc[[nanList[nn]]][['year','month','day','h'+str(hour-1)
                                           ,'h_'+str(hour-1),'h_'+str(hour)]]
                b= b.values.tolist()
#                 b[0][3]=int(str(b[0][3]).replace(',',''))
                my_data = b
#                 print(zone_id,hour,b)
                merge['h'+str(hour)][nanList[nn]]=int(mlr[zone_id][hour].predict(my_data)[0])
            else:
                b=merge.loc[[nanList[nn]]][['year','month','day','h'+str(hour-1)]]
                b= b.values.tolist()
#                 b[0][3]=int(str(b[0][3]).replace(',',''))
                my_data = b
                merge['h'+str(hour)][nanList[nn]]=int(mlr[zone_id][hour].predict(my_data)[0])


In [72]:
merge.to_csv("merge_predict.csv")